In [1]:
from pyro.infer import SVI, infer_discrete, TraceEnum_ELBO, TraceGraph_ELBO
import anneal.core
from anneal.models.MixtureGaussian import MixtureGaussian
from anneal.models.MixtureDirichlet import MixtureDirichlet
from anneal.models.MixtureGaussianDMP import MixtureGaussianDMP

import pyro
from pyro.optim import ClippedAdam
import anneal.utils 

from importlib import reload  

reload(anneal.core)
reload(anneal.utils)

interface = anneal.core.Interface()
interface.set_model(MixtureGaussianDMP)
interface.set_loss(TraceEnum_ELBO)
interface.set_optimizer(ClippedAdam)

data_dict = anneal.utils.load_simulation_seg("example1", "example1")
data_dict

{'data': tensor([[45., 36., 42.,  ..., 50., 47., 42.],
         [55., 55., 60.,  ..., 76., 88., 89.],
         [32., 14., 27.,  ..., 34., 32., 42.],
         ...,
         [43., 33., 43.,  ..., 53., 53., 51.],
         [13.,  3., 12.,  ..., 21., 19.,  9.],
         [28., 21., 40.,  ..., 45., 50., 46.]]),
 'pld': tensor([2.0000, 2.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 1.3000, 2.0000,
         2.0000, 1.0000, 1.7000, 2.0000, 2.0000, 3.0000, 2.0000, 2.0000, 2.0000,
         2.0000, 3.0000, 1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
         2.0000, 2.0000, 2.0000, 1.0000, 2.0000]),
 'segments': 32,
 'mu': tensor([12, 17, 14, 13,  8, 12, 29, 11, 12, 24, 15, 15, 12, 15, 11,  9, 13,  9,
         12, 26, 20, 17, 16,  8,  8, 15, 16, 13,  9, 11,  7,  9])}

In [2]:
pyro.enable_validation(True)
interface.initialize_model(data_dict)
interface.set_model_params({'alpha': 0.1})

loss = interface.run(200, MAP = True, param_optimizer={"lr" : 0.05 }, verbose=False
                     )

Running MixtureGaussianDMP on 1000 cells for 200 steps
..........

..........

Done!


In [3]:
lr = interface.learned_parameters(posterior=True, verbose=True)

Computing assignment probabilities
    0	117.3846640625
    1	117.142390625
    2	116.90196875
    3	116.6635625
    4	116.4272265625
    5	116.192984375
    6	115.960734375
    7	115.7304453125
    8	115.502046875
    9	115.275421875
   10	115.050515625
   11	114.8272890625
   12	114.6056484375
   13	114.385609375
   14	114.1672421875
   15	113.95059375
   16	113.7358359375
   17	113.5231328125
   18	113.31275
   19	113.1049453125
   20	112.9000546875
   21	112.698390625
   22	112.500328125
   23	112.306203125
   24	112.116390625
   25	111.9312109375
   26	111.7509765625
   27	111.57596875
   28	111.4063984375
   29	111.242453125
   30	111.084328125
   31	110.9320703125
   32	110.7857109375
   33	110.64534375
   34	110.5108671875
   35	110.382234375
   36	110.25934375
   37	110.1420625
   38	110.030265625
   39	109.9237109375
   40	109.8222734375
   41	109.7257421875
   42	109.633875
   43	109.5465078125
   44	109.463421875
   45	109.3844375
   46	109.30928125
   47	109.237828125
   4

In [5]:
reload(anneal.utils)
print(lr)
anneal.utils.mix_weights(
    lr['mixture_weights']
)

{'mixture_weights': array([0.01132488, 0.02300205, 0.02606214, 0.03652517, 0.3337265 ],
      dtype=float32), 'cnv_probs': array([[1.5270157 , 1.935663  , 0.9684033 , 1.8093878 , 1.9278655 ,
        1.813099  , 2.126808  , 0.9373788 , 1.577631  , 2.047112  ,
        0.85884774, 1.8297815 , 1.6846972 , 1.7171644 , 2.809366  ,
        1.6919596 , 2.2087195 , 2.1428182 , 1.8156369 , 2.8429954 ,
        0.98470676, 1.6768768 , 1.8534262 , 2.2266629 , 2.0560083 ,
        2.052784  , 1.7949954 , 1.847363  , 1.5395247 , 1.6904163 ,
        1.0155729 , 1.807066  ],
       [2.0076463 , 1.8532708 , 1.0767727 , 1.8186979 , 1.8263298 ,
        1.9539988 , 1.9473485 , 1.0016394 , 1.7956972 , 1.815759  ,
        0.9926646 , 1.6860161 , 2.0178676 , 1.9483515 , 3.05134   ,
        1.9490386 , 1.7304044 , 2.086009  , 1.9067436 , 2.7960622 ,
        0.83629984, 1.6621474 , 2.0541458 , 1.8191117 , 2.0008824 ,
        1.8577131 , 1.6550221 , 1.8163991 , 1.9062483 , 1.9068662 ,
        1.010959  , 2.149293

tensor([0.0113, 0.0227, 0.0252, 0.0344, 0.3025, 0.6039])